In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

import torchtext
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.data.metrics import bleu_score
from torchtext.vocab import build_vocab_from_iterator

import random
import numpy as np

import time
import math

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
# 设置随机种子
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
# 获取分词器
tokenizer_de = get_tokenizer('spacy', language='de_core_news_sm')
tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')

In [4]:
# 划分训练集、验证集和测试集
train_data, valid_data, test_data = Multi30k(root='NLP/dataset/Multi30k',
                                            split=('train', 'valid', 'test'), language_pair=('de', 'en'))

In [5]:
# 生成词表的迭代函数
def yield_tokens(dataset, language):
    for de, en in dataset:
        if language == 'de':
            yield [token.lower() for token in tokenizer_de(de)]
        elif language == 'en':
            yield [token.lower() for token in tokenizer_en(en)]
        else:
            raise KeyError

In [ ]:
# 构建词表
special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']
UNK_IDX = 0
PAD_IDX = 1
SOS_IDX = 2
EOS_IDX = 3

vocab_de = build_vocab_from_iterator(yield_tokens(train_data,'de'), min_freq=2, 
                                    specials=special_tokens, special_first=True)

vocab_en = build_vocab_from_iterator(yield_tokens(train_data,'en'), min_freq=2, 
                                    specials=special_tokens, special_first=True)

vocab_de.set_default_index(UNK_IDX)
vocab_en.set_default_index(UNK_IDX)

In [7]:
# 将句子进行分词，to_id，转为tensor
def transform_text(sentence, language):
    if language == 'de':
        # 分词
        sentence = [token.lower() for token in tokenizer_de(sentence)]
        # 转换为id
        sentence = [vocab_de[i] for i in sentence]
    elif language == 'en':
        sentence = [token.lower() for token in tokenizer_en(sentence)]
        sentence = [vocab_en[i] for i in sentence]
    else:
        raise KeyError
    # 转为tensor
    sentence = torch.cat((torch.tensor([SOS_IDX]), torch.tensor(sentence), torch.tensor([EOS_IDX])))
    return sentence

In [8]:
def collate_fn(data):
    batch_de, batch_en = [], []
    for de, en in data:
        de = transform_text(de, 'de')
        en = transform_text(en, 'en')
        batch_de.append(de)
        batch_en.append(en)
    batch_de = pad_sequence(batch_de, padding_value=PAD_IDX)
    batch_en = pad_sequence(batch_en, padding_value=PAD_IDX)

    return batch_de, batch_en

In [9]:
# 获取dataloader
train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True, drop_last=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_data, batch_size=128, shuffle=True, drop_last=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=True, drop_last=True, collate_fn=collate_fn)

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src_len, batch_size]
        # embedded = [src_len, batch_size, emb_size]
        embedded = self.dropout(self.embedding(src))
        # outputs = [src_len, batch_size, hid_dim * num_directions]
        # hidden = [n_layers * num_directions, batch_size, hid_dim]
        outputs, hidden = self.rnn(embedded)
        # features = [batch_size, hid_dim * 2]
        features = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        # hidden = [batch_size, dec_hid_dim]
        hidden = torch.tanh(self.fc(features))

        # outputs = [src_len, batch_size, hid_dim * 2] hidden = [batch_size, dec_hid_dim]
        return outputs, hidden

In [11]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=True)

    def forward(self, hidden, encoder_outputs):
        # hidden是query, encoder_outputs是src_len个key和value
        # hidden = [batch_size, dec_hid_dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        # hidden.unsqueeze(1) 变为 [batch_size, 1, dec_hid_dim]
        # hidden.unsqueeze(1).repeat(1, src_len, 1) 变为 [batch_size, src_len, dec_hid_dim]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        # encoder_outputs = [batch_size, src_len, enc_hid_dim * 2]
        encoder_outputs = encoder_outputs.permute(1,0,2)

        # feature = [batch_size, src_len, enc_hid_dim * 2 + dec_hid_dim]
        feature = torch.cat((hidden, encoder_outputs), dim = 2)
        # attention分数   energy = [batch_size, src_len, dec_hid_dim]
        energy = torch.tanh(self.attn(feature))
        
        # attention = [batch_size, src_len]
        attention = self.v(energy).squeeze(2)
        # 得到attention的分数
        return F.softmax(attention, dim=1)

In [12]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        # input是当前阶段的输入（一个word）
        # hidden是上个阶段的输出
        # encoder_outputs是encoder对每个word的输出

        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        input = input.unsqueeze(0)
        # [1, batch_size, emb_dim]
        embedded = self.dropout(self.embedding(input))

        # 得到注意力分散  a = [batch_size, src_len]
        a = self.attention(hidden, encoder_outputs)
        # a = [batch_size, 1, src_len]
        a = a.unsqueeze(1)

        # encoder_outputs = [batch_size, src_len, enc_hid_dim * 2]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # weighted = [batch_size, 1, enc_hid_dim * 2]
        weighted = torch.bmm(a, encoder_outputs)
        # weighted = [1, batch_size, enc_hid_dim * 2]
        weighted = weighted.permute(1, 0, 2)

        # rnn_input = [1, batch_size, emb_dim + enc_hid_dim * 2]
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        # 输入上阶段的hidden，得到当前阶段的hidden
        # 使用上阶段的hidden对GRU的hidden初始化
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)


In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)

        input = trg[0, :]
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1
        return outputs


In [14]:
input_dim = len(vocab_de)
output_dim = len(vocab_en)
enc_emb_dim = 256
dec_emb_dim = 256
enc_hid_dim = 512
dec_hid_dim = 512
enc_dropout = 0.2
dec_dropout = 0.2

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
attn = Attention(enc_hid_dim, dec_hid_dim)
encoder = Encoder(input_dim, enc_emb_dim, enc_hid_dim, dec_hid_dim, enc_dropout)
decoder = Decoder(output_dim, dec_emb_dim, enc_hid_dim, dec_hid_dim, dec_dropout, attn)

model = Seq2Seq(encoder, decoder, device).to(device)

In [15]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=True)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,518,406 trainable parameters


In [17]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [18]:
def train(model, dataloader, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, (src, trg) in enumerate(dataloader):
        src = src.to(device)
        trg = trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(list(dataloader))

In [19]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, (src, trg) in enumerate(iterator):
            src = src.to(device)
            trg = trg.to(device)
            output = model(src, trg, 0) #turn off teacher forcing
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(list(iterator))

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'rnn_with_attention.pt')
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 46s
	Train Loss: 5.025 | Train PPL: 152.107
	 Val. Loss: 4.830 |  Val. PPL: 125.209
Epoch: 02 | Time: 0m 46s
	Train Loss: 4.156 | Train PPL:  63.823
	 Val. Loss: 4.464 |  Val. PPL:  86.862
Epoch: 03 | Time: 0m 45s
	Train Loss: 3.571 | Train PPL:  35.569
	 Val. Loss: 3.874 |  Val. PPL:  48.144
Epoch: 04 | Time: 0m 45s
	Train Loss: 2.944 | Train PPL:  18.997
	 Val. Loss: 3.498 |  Val. PPL:  33.051
Epoch: 05 | Time: 0m 45s
	Train Loss: 2.485 | Train PPL:  12.002
	 Val. Loss: 3.313 |  Val. PPL:  27.475
Epoch: 06 | Time: 0m 45s
	Train Loss: 2.143 | Train PPL:   8.525
	 Val. Loss: 3.294 |  Val. PPL:  26.958
Epoch: 07 | Time: 0m 45s
	Train Loss: 1.878 | Train PPL:   6.539
	 Val. Loss: 3.274 |  Val. PPL:  26.407
Epoch: 08 | Time: 0m 47s
	Train Loss: 1.647 | Train PPL:   5.192
	 Val. Loss: 3.346 |  Val. PPL:  28.392
Epoch: 09 | Time: 0m 45s
	Train Loss: 1.469 | Train PPL:   4.347
	 Val. Loss: 3.394 |  Val. PPL:  29.781
Epoch: 10 | Time: 0m 45s
	Train Loss: 1.307 | Train PPL

In [22]:
# 评估
test_loss = evaluate(model, test_dataloader, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.387 | Test PPL:  29.590 |


In [23]:
# 翻译句子
def translation(model, iterator):
    model.eval()
    epoch_loss = 0
    preditions = []
    references = []
    en_vocab = vocab_en.get_itos()
    with torch.no_grad():
        for i, (src, trg) in enumerate(iterator):
            src = src.to(device)
            trg = trg.to(device)
            
            output = model(src, trg, 0)
            output_dim = output.shape[-1]
            output = output[1:].permute(1,0,2)
            output = output.argmax(dim=-1)

            trg = trg.permute(1,0)
            trg = trg[:,1:]

            batch_size = trg.shape[0]
            
            for i in range(batch_size):
                sentence = []
                for j in trg[i]:
                    if en_vocab[j] == '<eos>':
                        break
                    sentence.append(en_vocab[j])
                references.append([sentence])
            
            for i in range(batch_size):
                sentence = []
                for j in output[i]:
                    if en_vocab[j] == '<eos>':
                        break
                    sentence.append(en_vocab[j])
                preditions.append(sentence)
    return preditions, references

In [24]:
preditions, references = translation(model, test_dataloader)
bleu_score(preditions, references)

0.3199716799284989

In [25]:
' '.join(preditions[1]), ' '.join(references[1][0])

('two muddy - dogs are running across the snow .',
 'two medium sized dogs run across the snow .')